<a href="https://colab.research.google.com/github/mauroschramm/aprendizado-de-m-quina-2---furb/blob/main/Eigen_Faces_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Aprendizado de Máquina Não Supervisionado
# Reconhecimento de faces da base de imagens ORL
#
# Autor: Mauro Schramm
#
import cv2, os, sys, random, copy
import numpy as np
from google.colab.patches import cv2_imshow

# classe Person - reprsenta as pessoas cujas imagens estão no repositório
class Person:
  pass


# Classe PCA
# classe utilitária com vários métodos estáticos que proveem seriços auxiliares
class PCA:

  # get_image_data - carrega uma imagem a partir de um arquivo e a tranforma em
  #                tons de cinza com 80x80 pixels com números de ponto flutuante
  @staticmethod
  def get_image_data(filename):
    path_file = filename
    img = cv2.imread(path_file, cv2.IMREAD_GRAYSCALE)

    dst = cv2.resize(img, (80, 80))

    #transformar a imagem em uma matriz coluna
    #  r: opencv em Python utiliza a classe numpy.ndarray para objetos cv::Mat. 
    #     O método reshape de numpy.ndarray possui assinatura diferente do mesmo método em cv::Mat 
    #     Não utilizado, pois não foi feita implementação da PCA, e sim utilizada biblioteca OpenCV 
    #dst = dst.T.reshape((dst.shape[0] * dst.shape[1], 1))

    data = np.float64(dst)

    return data

  # to_person - cria um objeto da classe Person a partir de um arquivo de imagem
  @staticmethod
  def to_person( filename ):

    filename_proc = filename.lower()
    filename_proc = filename_proc.rstrip(".jpg")
    filename_proc = filename_proc[(filename_proc.rfind("/") + 1) : (len(filename_proc))]

    person = Person();
    person.id = int(filename_proc.split("_")[0])
    person.label = int(filename_proc.split("_")[1])
    person.data = PCA.get_image_data(filename)

    return person

  # load_data_set - carrega os arquivos de imagem ORL como objetos Person e separa
  #                 os separa em conjuntos de treino e teste
  @staticmethod
  def load_data_set(path, train, test, p):

    # lista de arquivos do diretório
    dirs = os.listdir(path)
    i = 0
    while True:
      if not dirs[i].lower().endswith(".jpg"):
        del dirs[i]
      i = i + 1
      if i >= len(dirs):
        break
    
    people = []
    for file in dirs:
      person = PCA.to_person(path + file)
      people.append(person)

    people.sort(key=lambda p: p.id)
      
    num_samples_per_person = 10
    samples = []
    for person in people:
      samples.append(person)

      if len(samples) == num_samples_per_person:
        
        while len(samples) > p:
          index = random.randint(0, len(samples)-1)
          test.append(samples[index])
          del samples[index]

        if p == num_samples_per_person:
          test.extend(samples)

        train.extend(samples)
        samples.clear()

#
# Ponto de inicio da execução
#
path = "./"
train=[]
test=[]
p=7

PCA.load_data_set(path, train, test, p)

start = 10
end = 20

for k in range (start, end+1):
  model = cv2.face.EigenFaceRecognizer_create(k)
  src= []
  labels = np.zeros((len(train),1), dtype=np.int32)

  for i in range(0, len(train)):
    person = train[i]
    src.append(person.data)
    labels[i,0] = person.label

  model.train(src, labels)

  tries = 0;
  corrects = 0;

  for person_test in test:
    test_data = person_test.data
    label, confidence = model.predict(test_data)

    tries = tries + 1
    if label == person_test.label: corrects = corrects + 1
      
  print(k, "componentes principais, acurácia", "{:.0%}".format(corrects/tries))
  


10 componentes principais, acurácia 95%
11 componentes principais, acurácia 96%
12 componentes principais, acurácia 97%
13 componentes principais, acurácia 98%
14 componentes principais, acurácia 97%
15 componentes principais, acurácia 97%
16 componentes principais, acurácia 97%
17 componentes principais, acurácia 98%
18 componentes principais, acurácia 97%
19 componentes principais, acurácia 97%
20 componentes principais, acurácia 98%
